In [3]:
import pandas as pd
import os
import numpy as np
import pandas_profiling

# This line is needed to display plots inline in Jupyter Notebook
%matplotlib inline

# Required for basic python plotting functionality
import matplotlib.pyplot as plt

# Required for formatting dates later in the case
import datetime
import matplotlib.dates as mdates

# Required to display image inline
from IPython.display import Image

# Advanced plotting functionality with seaborn
import seaborn as sns
sns.set(style="whitegrid") # can set style depending on how you'd like it to look

import folium  #needed for interactive map
from folium.plugins import HeatMap

import geopandas
from shapely.geometry import Polygon
from shapely.geometry import Point
from geopandas import GeoDataFrame

In [5]:
path = 'Dataset/Original'
zones = pd.read_csv(path + '/zones.csv', sep = ',', doublequote = True)

# Opens the map as a GeoDataFrame
path = 'Dataset/Original'
nyc_bouroughs = geopandas.read_file(path + '/Neighborhood Tabulation Areas.geojson')

path = 'Dataset/Parquets'
uber_trips_2 = pd.read_parquet(path + '/uber2015_trips_filtered.parquet', engine = 'pyarrow')

In [7]:
uber_trips_f = uber_trips_2.merge(zones, how = 'inner', left_on = 'pickup_location_id', right_on='location_id')
uber_trips_f.head()

,pickup_datetime,pickup_location_id,dispatch_base,affiliate_base,pickup_month,location_id,borough,zone,service_zone,nta_code
0,2015-05-17 09:47:00,141,B02617,B02617,201505,141,Manhattan,Lenox Hill West,Yellow Zone,MN31
1,2015-05-17 09:50:00,141,B02617,B02617,201505,141,Manhattan,Lenox Hill West,Yellow Zone,MN31
2,2015-05-17 09:53:00,141,B02617,B02617,201505,141,Manhattan,Lenox Hill West,Yellow Zone,MN31
3,2015-05-17 10:00:00,141,B02617,B02617,201505,141,Manhattan,Lenox Hill West,Yellow Zone,MN31
4,2015-05-17 10:00:00,141,B02617,B02617,201505,141,Manhattan,Lenox Hill West,Yellow Zone,MN31


In [30]:
print(uber_trips_f.shape)
uber_trips_f_nta = uber_trips_f.merge(nyc_bouroughs, how = 'inner', left_on = 'nta_code', right_on = 'ntacode')
print(uber_trips_f_nta.shape)

(14264215, 10)
(14264110, 18)


In [32]:
uber_trips_f_nta.drop(['location_id','borough','zone','service_zone','nta_code'], axis = 1, inplace = True)
uber_trips_f_nta.head()

,pickup_datetime,pickup_location_id,dispatch_base,affiliate_base,pickup_month,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code,geometry
0,2015-05-17 09:47:00,141,B02617,B02617,201505,MN31,21508615.8863,061,Lenox Hill-Roosevelt Island,39424.2256525,Manhattan,1,"MULTIPOLYGON (((-73.95764 40.74841, -73.95749 ..."
1,2015-05-17 09:50:00,141,B02617,B02617,201505,MN31,21508615.8863,061,Lenox Hill-Roosevelt Island,39424.2256525,Manhattan,1,"MULTIPOLYGON (((-73.95764 40.74841, -73.95749 ..."
2,2015-05-17 09:53:00,141,B02617,B02617,201505,MN31,21508615.8863,061,Lenox Hill-Roosevelt Island,39424.2256525,Manhattan,1,"MULTIPOLYGON (((-73.95764 40.74841, -73.95749 ..."
3,2015-05-17 10:00:00,141,B02617,B02617,201505,MN31,21508615.8863,061,Lenox Hill-Roosevelt Island,39424.2256525,Manhattan,1,"MULTIPOLYGON (((-73.95764 40.74841, -73.95749 ..."
4,2015-05-17 10:00:00,141,B02617,B02617,201505,MN31,21508615.8863,061,Lenox Hill-Roosevelt Island,39424.2256525,Manhattan,1,"MULTIPOLYGON (((-73.95764 40.74841, -73.95749 ..."


In [33]:
path = 'Dataset/Parquets'
uber_trips_f_nta.drop('geometry', axis = 1, inplace = True)
uber_trips_f_nta.to_parquet(path + '/uber2015_trips_nta.parquet', engine = 'pyarrow')